In [1]:
import numpy as np
import os
import time
import glob

In [ ]:
## Taken form the SAMoSA -Analysis library developed by Silke Henkes
## Refer for further details https://github.com/silkehenkes/SAMoSA
## Dated : June 2025

import pandas
import gzip

class ReadData:
  
	def __init__(self, filename,dialect):
		if filename.split('.')[-1] == 'gz':
			names = ['id', 'type', 'flag', 'radius', 'x','y','z', 'vx', 'vy', 'vz', 'nx', 'ny', 'nz']
			self.datafile = gzip.open(filename, newline='')
			line = self.datafile.readline()
			self.header_names = 0
			if line.startswith("#"):
				self.header_names = line[1:].strip().split()
			self.datafile.seek(0)	

		else:
			self.datafile = open(filename, newline='')
			line = self.datafile.readline()
			self.header_names = 0
			names = ['id', 'type', 'flag', 'radius', 'x','y','z', 'vx', 'vy', 'vz', 'nx', 'ny', 'nz']
			if line.startswith("#"):
				self.header_names = line[1:].strip().split()
			else:
				self.header_names = names
			self.datafile.seek(0)	
		self.dialect = dialect
		self.__read_data()

	

	# Read data using pandas. Simplify data structure for Configuration
	def __read_data(self):
		if self.dialect == "SAMoS":
			
			self.data = pandas.read_csv(self.datafile,sep=r"\s+", comment= "#", names = self.header_names)
			#
			# temp = self.data.columns
			#colshift = {}
			#for u in range(len(temp)-1): 
			#	colshift[temp[u]] = temp[u+1]
			#self.data.rename(columns = {temp[len(temp)-1]: 'garbage'},inplace=True)
			#self.data.rename(columns = colshift,inplace=True,errors="raise")
			#print(self.data.columns)
		elif self.dialect == "CCCPy":
			self.data = pandas.read_csv(self.datafile,header=0)
			# look of the header
			# currTime,xPos,yPos,xVel,yVel,polAngle,polVel,xPol,yPol,rad,glued
			# We need to muck about with the headers to distil this to a unified format
			# Classical samos header:
			#  id  type  flag  radius  x  y  z  vx  vy  vz  nx  ny  nz 
			self.data.rename(columns={"xPos": "x", "yPos": "y", "xVel": "vx", "yVel": "vy", "xPol": "nx", "yPol": "ny", "rad":"radius", "glued":"type"}, inplace=True,errors="raise")
			#print(self.data.columns)
		elif self.dialect == "CAPMD":
			self.data = pandas.read_csv(self.datafile,header=0)
		else:
			print("Unknown data format dialect!")
		


In [31]:
def extract_positions(files_path,tp):
    files = sorted(glob.glob(files_path))
    

    positions = []

    for file in files:   
        read_file = ReadData(file, "SAMoS")
        read_data= read_file.data
        data1 = read_data[read_data['type']==tp]
        cur_positions = np.stack((data1['x'], data1['y'], data1['z'])).T
        positions.append(cur_positions)
    
    positions = np.asarray(positions)
    return positions

In [4]:
def extract_vel(files_path):
    files = sorted(glob.glob(files_path))

    velocities = []

    for file in files:   
        read_file = ReadData(file, "SAMoS")
        data1= read_file.data
        cur_velocities = np.stack((data1['vx'], data1['vy'], data1['vz'])).T
        velocities.append(cur_velocities)
    
    velocities = np.asarray(velocities)
    return velocities


In [5]:
def extract_directors(files_path):
    files = sorted(glob.glob(files_path))

    directors = []

    for file in files:   
        read_file = ReadData(file, "SAMoS")
        data1= read_file.data
        cur_directors = np.stack((data1['nx'], data1['ny'], data1['nz'])).T
        directors.append(cur_directors)
    
    directors = np.asarray(directors)
    return directors

In [6]:
def extract_particle_radii(files_path):
    files = sorted(glob.glob(files_path))
    radii = []
    for file in files:   
        read_file = ReadData(file, "SAMoS")
        data1= read_file.data
        cur_radii = data1['radius']
        radii.append(cur_radii)
    
    radii = np.asarray(radii)
    return radii

In [7]:
def radius_of_gyration(positions):
    
    N = np.shape(positions)[0]
    sum_r = np.sum(positions, axis = 0)
    sum_r2 = np.sum(np.sum(positions**2, axis=1))
    rg_2 = sum_r2/N - np.sum(sum_r**2)/N**2
    return np.sqrt(rg_2)

In [8]:
def center_of_mass(positions):
    N = np.shape(positions)[0]
    sum_r = np.sum(positions,axis = 0)
    return sum_r/N

In [ ]:
output_file_path = "/data1/pabshettiwar/Simulation_Softwares/SAMOS_ABP/tumoroid_system_alignments/1000-morse/pair_nematic/Seed-5/xi_1.0_J_0.5_dr_0.01_abp-p_0.3_ma_2.50_mD_0.08/output_*.dat"
dir_path = '/data1/pabshettiwar/Simulation_Softwares/SAMOS_ABP/tumoroid_system_alignments/1000-morse/pair_nematic/Seed-5/xi_1.0_J_0.5_dr_0.01_abp-p_0.3_ma_2.50_mD_0.08/'

files = sorted(glob.glob(output_file_path))

positions_ts = []

for i, file in enumerate(files):   
    read_file = ReadData(file, "SAMoS")
    data_read= read_file.data

    # extract type 1 particle
    data_1 = data_read[data_read['type']==1]
    position_cur = np.stack((data_1['x'], data_1['y']), axis = 1)    
    positions_ts.append(position_cur)


In [10]:
import alphashape
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from shapely.plotting import plot_polygon

def get_shape(points,alpha):
    alpha_shape = alphashape.alphashape(points, alpha)
    #area = alpha_shape.area
    #perimeter = alpha_shape.length

    return alpha_shape #, area, perimeter

In [11]:
def plot_cells(points):
    # takes in the total points [(x,y), .. ] as arguments to plot
    
    x_points = points[0]
    y_points = points[1]

    fig,ax = plt.subplots(constrained_layout=True)
    ax.scatter(x_points,y_points)
    ax.set_aspect('equal', adjustable='box')
    plt.gca().set_aspect('equal')
    plt.show()

In [35]:
from pathlib import Path

def plot_alphashape(shape, points,frame = 1, outer_points= False,  dir = Path.cwd()):
    # arguments : alpha shape, original points, total number of time frames, directory to save output files(images) 
    fig, ax = plt.subplots(constrained_layout=True)
    ax.scatter(points.T[0], points.T[1], s=5)
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(-50,50)
    ax.set_ylim(-50,50)
    plot_polygon(shape, ax=ax, add_points=outer_points, facecolor='orange', alpha = 0.5)
    plt.savefig(os.path.join(dir, f"tumor_alphashape_test_{frame:04d}.png"), dpi = 100, format = 'png', )
    plt.close()
    return None

In [13]:
def get_shape_index(peri, area):
    # shape index measures closeness to circle
    # si = perimeter/ 2* (pi* area)**1/2 
    si = peri / (2* np.sqrt(np.pi * area))
    return si

In [28]:
'''
def generate_traj_shapes(traj, alpha, dir = dir_path):

    for frame in range(np.shape(traj)[0]):
        points_cur = traj[frame]
        a_shape = get_shape(points_cur, alpha)
        # a_shape, area, perimeter = get_shape(points_cur, alpha)
        plot_alphashape(a_shape, points_cur, frame = frame, outer_points= False, dir = os.path.join(dir_path, 'images/') ) 

'''


"\ndef generate_traj_shapes(traj, alpha, dir = dir_path):\n\n    for frame in range(np.shape(traj)[0]):\n        points_cur = traj[frame]\n        a_shape = get_shape(points_cur, alpha)\n        # a_shape, area, perimeter = get_shape(points_cur, alpha)\n        plot_alphashape(a_shape, points_cur, frame = frame, outer_points= False, dir = os.path.join(dir_path, 'images/') ) \n\n"

In [14]:
def generate_traj_shapes(traj, alpha, dir):

    for frame in range(np.shape(traj)[0]):
        points_cur = traj[frame]
        a_shape = get_shape(points_cur, alpha)
        # a_shape, area, perimeter = get_shape(points_cur, alpha)
        plot_alphashape(a_shape, points_cur, frame = frame, outer_points= False, dir = dir ) 


In [37]:
data_dir_path = '/data1/pabshettiwar/Simulation_Softwares/SAMOS_ABP/tumoroid_system_alignments/1000-morse/pair_polar/Seed-5'
alpha = 0.5

for dir in os.listdir(data_dir_path):

    cur_path = os.path.join(data_dir_path, dir)
    
    cur_output_files = os.path.join(cur_path, 'output_*.dat')
    
    cur_traj = extract_positions(cur_output_files, tp=1)

    if not os.path.isdir(os.path.join(cur_path, 'alpha_images')):
        os.mkdir(os.path.join(cur_path, 'alpha_images'))
    
    img_path = os.path.join(cur_path, 'alpha_images')
    
    generate_traj_shapes(cur_traj[101:,:,:2], alpha, dir = img_path )

In [39]:
def MSD(positions:np.ndarray):
    # simple msd calculations ==> |x(t) - x(0)|**2 for frame(t) and averaged over all particles for each frame i.e ensemble average
    # there is no moving tau window used here  
    msd = np.mean(np.sum((positions - positions[0,:,:][np.newaxis,:,:])**2, axis = 2), axis = 1)
    return msd


In [43]:
data_dir_path = '/data1/pabshettiwar/Simulation_Softwares/SAMOS_ABP/tumoroid_system_alignments/1000-morse/pair_polar/Seed-5'

for dir in os.listdir(data_dir_path):

    cur_path = os.path.join(data_dir_path, dir)

    cur_output_files = os.path.join(cur_path, 'output_*.dat')

    cur_traj = extract_positions(cur_output_files, tp=1)

    msd_cur = MSD(cur_traj[101:])

    rog_all = []

    for i in range(np.shape(cur_traj)[0]):
        rog = radius_of_gyration(cur_traj[i])
        rog_all.append(rog)

    fig,ax = plt.subplots()
    ax.loglog(msd_cur)
    plt.savefig(os.path.join(cur_path, f"msd_c.png"), format= "png", dpi = 200)
    plt.close()

    fig,ax = plt.subplots()
    ax.plot(rog_all)
    plt.savefig(os.path.join(cur_path,f"Radius_of_gyration_c.png"), format="png", dpi = 200)
    plt.close()
